# **Import (AWS S3) , Cleansing and Transform Covid19 cases data using PySpark**

**Steps**
1. Import Covid19 dataset from AWS S3 (using datasource from data.go.th)
2. Drop uninterested columns
3. Handle with missing values
4. Create a new column to group age by range
5. Find the wrong or inconsistent Province name in the dataset and correct them
6. Save output as parquet file
7. Send notification to users that data is ready using AWS SNS

# Install libraries to use

In [2]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 198 kB 67.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=1d26e0a3e391f0994d349a04c8f29c3c9f4d7c4634d5b5da60f7b37eb4d5d084
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
pip install boto3

     |████████████████████████████████| 131 kB 4.9 MB/s 
     |████████████████████████████████| 7.8 MB 36.1 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 138 kB 61.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.6 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Import datasets from AWS S3

In [36]:
import boto3

s3 = boto3.client('s3', aws_access_key_id = AWS_ACCESS_KEY, aws_secret_access_key = AWS_SECRET_KEY)
s3.download_file(Bucket='mj-dataset', Filename='datasets/covid19-case.csv', Key='covid19-case.csv')
s3.download_file(Bucket='mj-dataset', Filename='datasets/province.csv', Key='province.csv')

# Import libraries and create SparkSession

In [14]:
from pyspark.sql import  SparkSession, Row, Column
from pyspark.sql.functions import regexp_replace, col, count, when, isnan

spark = SparkSession.builder.getOrCreate()

# Exploration, Cleansing and Transformation of Covid19 cases data

In [15]:
# Import Covid19 cases data
df_covid = spark.read.csv('datasets/covid19-case.csv', header=True, inferSchema=True)

In [16]:
# Explore data
df_covid.printSchema()

df_covid.show(5)

df_covid.summary().show()

root
 |-- No.: string (nullable = true)
 |-- announce_date: string (nullable = true)
 |-- Notified date: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- Unit: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- province_of_isolation: string (nullable = true)
 |-- risk: string (nullable = true)
 |-- province_of_onset: string (nullable = true)
 |-- district_of_onset: string (nullable = true)

+---+-------------+-------------+----+----+----+-----------+---------------------+--------------------+-----------------+-----------------+
|No.|announce_date|Notified date| sex| age|Unit|nationality|province_of_isolation|                risk|province_of_onset|district_of_onset|
+---+-------------+-------------+----+----+----+-----------+---------------------+--------------------+-----------------+-----------------+
|  1|   12/01/2020|         null|หญิง|61.0|  ปี|      China|        กรุงเทพมหานคร|คนต่างชาติเดินทาง...|    กรุงเท

In [17]:
# Drop unused columns
df_covid = df_covid.drop('No.', 'Notified date', 'district_of_isolation', 'province_of_isolation', 'district_of_onset')
print(df_covid.show(5))

# Check missing values of all columns
df_covid.select( [ count( when(col(c).isNull() | isnan(col(c)), c) ).alias(c) for c in df_covid.columns]).show()

+-------------+----+----+----+-----------+--------------------+-----------------+
|announce_date| sex| age|Unit|nationality|                risk|province_of_onset|
+-------------+----+----+----+-----------+--------------------+-----------------+
|   12/01/2020|หญิง|61.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|
|   17/01/2020|หญิง|74.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|
|   22/01/2020|หญิง|73.0|  ปี|   Thailand|คนต่างชาติเดินทาง...|           นครปฐม|
|   22/01/2020| ชาย|68.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|
|   24/01/2020|หญิง|66.0|  ปี|      China|คนต่างชาติเดินทาง...|          นนทบุรี|
+-------------+----+----+----+-----------+--------------------+-----------------+
only showing top 5 rows

None
+-------------+-----+-----+-----+-----------+----+-----------------+
|announce_date|  sex|  age| Unit|nationality|risk|province_of_onset|
+-------------+-----+-----+-----+-----------+----+-----------------+
|           10|22175|3438

In [18]:
# Filter only row where age and province_of_onset is not null
df_covid_filtered = df_covid.where( col('age').isNotNull() & col('Unit').isNotNull() & col('province_of_onset').isNotNull()  )

# Verified that column age and province_of_onset have no missing values
df_covid_filtered.select( [ count( when(col(c).isNull() | isnan(col(c)), c) ).alias(c) for c in df_covid.columns]).show()

+-------------+----+---+----+-----------+----+-----------------+
|announce_date| sex|age|Unit|nationality|risk|province_of_onset|
+-------------+----+---+----+-----------+----+-----------------+
|            0|8891|  0|   0|      17845| 924|                0|
+-------------+----+---+----+-----------+----+-----------------+



In [19]:
# Create a new column named age_range for grouping age by range
df_covid_group_age = df_covid_filtered.withColumn('age_range', when( (df_covid_filtered.Unit=='วัน')  | (df_covid_filtered.Unit=='เดือน')  | ( (df_covid_filtered.Unit=='ปี') & (df_covid_filtered.age <=5) )  , '0-5 ปี') 
                                                  .when( (df_covid_filtered.Unit=='ปี') & (df_covid_filtered.age >= 6) & (df_covid_filtered.age <=17) , '6-17 ปี') 
                                                  .when( (df_covid_filtered.Unit=='ปี') & (df_covid_filtered.age >= 18) & (df_covid_filtered.age <= 30), '18-30 ปี') 
                                                  .when( (df_covid_filtered.Unit=='ปี') & (df_covid_filtered.age >= 31) & (df_covid_filtered.age <= 60), '31-60 ปี') 
                                                  .otherwise('มากกว่า 60 ปี') )
df_covid_group_age.show(10)

+-------------+----+----+----+-----------+--------------------+-----------------+-------------+
|announce_date| sex| age|Unit|nationality|                risk|province_of_onset|    age_range|
+-------------+----+----+----+-----------+--------------------+-----------------+-------------+
|   12/01/2020|หญิง|61.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|
|   17/01/2020|หญิง|74.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|
|   22/01/2020|หญิง|73.0|  ปี|   Thailand|คนต่างชาติเดินทาง...|           นครปฐม|มากกว่า 60 ปี|
|   22/01/2020| ชาย|68.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|
|   24/01/2020|หญิง|66.0|  ปี|      China|คนต่างชาติเดินทาง...|          นนทบุรี|มากกว่า 60 ปี|
|   25/01/2020|หญิง|33.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|     31-60 ปี|
|   26/01/2020|หญิง|57.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|     31-60 ปี|
|   26/01/2020|หญิง|73.0|  ปี|      Chin

In [20]:
# Check unique value of Province name
df_covid_group_age.select('province_of_onset').distinct().count()

81

In [23]:
# Since total number of Province need to be less than or equal 77, so we need to cross check with Province name list which collect in province.csv
df_province = spark.read.csv('datasets/province.csv', header=True, inferSchema=True)

In [24]:
# Check total number of Province name and Explore schema
print(f'Total province = {df_province.distinct().count()}')

df_province.printSchema()

Total province = 77
root
 |-- ProvinceNo: integer (nullable = true)
 |-- ProvinceMOI_ID: integer (nullable = true)
 |-- ProvinceNameThai: string (nullable = true)
 |-- ProvinceNameThaiAbbr: string (nullable = true)
 |-- ProvinceBudgetCode_2563: integer (nullable = true)
 |-- ProvinceNameEnglish: string (nullable = true)
 |-- ProvinceOfficialWebsite: string (nullable = true)
 |-- TopExecutivePositionandProvinceName: string (nullable = true)
 |-- AddressforCorrespondence: string (nullable = true)
 |-- RegionID: integer (nullable = true)
 |-- RegionName: string (nullable = true)
 |-- ProvincialClusterID: integer (nullable = true)
 |-- ProvincialClusterName: string (nullable = true)
 |-- RegionCGDWelfareIntegrationSystem: string (nullable = true)
 |-- RegionTMD: string (nullable = true)
 |-- RegionECT: string (nullable = true)
 |-- RegionNSO: string (nullable = true)
 |-- RegionOPSIndustry: string (nullable = true)
 |-- RegionArmy: string (nullable = true)
 |-- RegionPolice: string (nullab

In [25]:
# Select only the ProvinceNameThai column, rename to Province column and remove word 'จังหวัด' from province name
df_province_name = df_province.select('ProvinceNameThai').withColumn('province', regexp_replace('ProvinceNameThai', 'จังหวัด', '')).drop('ProvinceNameThai')

# Keep Province names as list
provinces =[row.province for row in df_province_name.collect()]
print( len(provinces) )

77


In [26]:
# Find incorrect or inconsistent Province name from Covid19 data
df_covid_group_age.select('province_of_onset').distinct().filter( ~ df_covid_group_age['province_of_onset'].isin(provinces) ).show()

+-----------------+
|province_of_onset|
+-----------------+
|         กรุงเทพฯ|
|            โคราช|
|             กทม.|
|          กรุงเทพ|
+-----------------+



In [27]:
# Replace inconsistent Province name with the correct name
df_covid_final = df_covid_group_age.withColumn('province',  when(( df_covid_group_age['province_of_onset']=='กรุงเทพฯ') | (df_covid_group_age['province_of_onset']=='กรุงเทพ') \
                                                                 | (df_covid_group_age['province_of_onset']=='กทม.') , 'กรุงเทพมหานคร')  \
                                                                 .when( df_covid_group_age['province_of_onset']=='โคราช', 'นครราชสีมา')
                                                                  .otherwise( df_covid_group_age['province_of_onset'])  )

# Verify that total number of Province is 77
df_covid_final.select('province').distinct().count()

77

In [28]:
# Final Output
df_covid_final.show()

+-------------+----+----+----+-----------+--------------------+-----------------+-------------+---------------+
|announce_date| sex| age|Unit|nationality|                risk|province_of_onset|    age_range|       province|
+-------------+----+----+----+-----------+--------------------+-----------------+-------------+---------------+
|   12/01/2020|หญิง|61.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|  กรุงเทพมหานคร|
|   17/01/2020|หญิง|74.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|  กรุงเทพมหานคร|
|   22/01/2020|หญิง|73.0|  ปี|   Thailand|คนต่างชาติเดินทาง...|           นครปฐม|มากกว่า 60 ปี|         นครปฐม|
|   22/01/2020| ชาย|68.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|  กรุงเทพมหานคร|
|   24/01/2020|หญิง|66.0|  ปี|      China|คนต่างชาติเดินทาง...|          นนทบุรี|มากกว่า 60 ปี|        นนทบุรี|
|   25/01/2020|หญิง|33.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|     31-60 ปี|  กรุงเทพ

In [32]:
# Save DataFrame to Parquet File
df_covid_final.write.parquet('covid_final', mode='overwrite')

In [33]:
# Test Output File
df = spark.read.parquet('covid_final')
df.show()

+-------------+----+----+----+-----------+--------------------+-----------------+-------------+---------------+
|announce_date| sex| age|Unit|nationality|                risk|province_of_onset|    age_range|       province|
+-------------+----+----+----+-----------+--------------------+-----------------+-------------+---------------+
|   12/01/2020|หญิง|61.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|  กรุงเทพมหานคร|
|   17/01/2020|หญิง|74.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|  กรุงเทพมหานคร|
|   22/01/2020|หญิง|73.0|  ปี|   Thailand|คนต่างชาติเดินทาง...|           นครปฐม|มากกว่า 60 ปี|         นครปฐม|
|   22/01/2020| ชาย|68.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|มากกว่า 60 ปี|  กรุงเทพมหานคร|
|   24/01/2020|หญิง|66.0|  ปี|      China|คนต่างชาติเดินทาง...|          นนทบุรี|มากกว่า 60 ปี|        นนทบุรี|
|   25/01/2020|หญิง|33.0|  ปี|      China|คนต่างชาติเดินทาง...|    กรุงเทพมหานคร|     31-60 ปี|  กรุงเทพ

# Send Notification to users via AWS SNS

In [41]:
sns = boto3.client('sns', region_name='us-east-2' , aws_access_key_id = AWS_ACCESS_KEY, aws_secret_access_key = AWS_SECRET_KEY)
sns.publish(TopicArn='arn:aws:sns:us-east-2:578712631654:alert_covid19_data', \
            Subject='Covid19 output file is ready', \
            Message='...Data has been cleaned and transformed...')

{'MessageId': '828e0d2e-fa88-55a8-8daa-54ba46a1efde',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '294',
   'content-type': 'text/xml',
   'date': 'Mon, 16 Aug 2021 19:18:00 GMT',
   'x-amzn-requestid': 'ba1fabef-0368-589b-9dd7-f27bbf4182ed'},
  'HTTPStatusCode': 200,
  'RequestId': 'ba1fabef-0368-589b-9dd7-f27bbf4182ed',
  'RetryAttempts': 0}}